In [ ]:
%%capture
import ROOT
import glob
import math
import sys, os
import numpy as np
import pandas as pd
from IPython.display import display, Markdown, HTML
import ipywidgets as widgets
from TPCQCVis.src.drawHistograms import *
from TPCQCVis.src.drawTrending import *
from TPCQCVis.src.drawMultiTrending import *
from TPCQCVis.src.checkHistograms import *
from TPCQCVis.src.checkTrending import *
from TPCQCVis.src.utility import *
import warnings
from copy import copy
warnings.filterwarnings('ignore')

In [ ]:
%jsroot on
display(HTML("<style>.container { width:95% !important; align-items: center;}</style>"))
display(HTML("<style>table {float:left;}</style>"))
ROOT.gErrorIgnoreLevel = ROOT.kError

CODEDIR = os.environ['TPCQCVIS_DIR']
DATADIR = os.environ['TPCQCVIS_DATA']
REPORTDIR = os.environ['TPCQCVIS_REPORT']
#display(HTML('<style>{}</style>'.format(CSS)))
#ROOT.gStyle.SetPalette(57)

In [ ]:
def drawQuantileProjection(objectName,rootDataFile,quantileOrder=0.5, axis="x"):
    from copy import copy
    canvas = ROOT.TCanvas()
    legend = ROOT.TLegend()
    [hist,leg,canvo,pad1] = drawHistograms(objectName,rootDataFile,normalize=False,legend=False,
                                               legendNames=runList,pads=True,drawOption="COLZ",maxColumns=3)
    quants = []
    canvas.cd()
    ROOT.gPad.SetGridy(1)
    for i,histo in enumerate(hist):
        if axis == "y":
            quant = copy(histo.QuantilesY(quantileOrder))
        else:
            quant = copy(histo.QuantilesX(quantileOrder))
        quant.SetYTitle("Median "+ histo.GetYaxis().GetTitle())
        quant.SetXTitle(histo.GetXaxis().GetTitle())
        quant.SetTitle(quant.GetYaxis().GetTitle()+" vs. "+quant.GetXaxis().GetTitle())
        quant.SetLineWidth(2)
        quants.append(quant)
        quants[i].SetStats(0)
        quants[i].Draw("SAME L")
        legend.AddEntry(quant,runList[i])
    return quants,legend,canvas
    
def sliceAndFit(objectName,rootDataFile,fitFunc="gaus",fitRange=[40,60]):
    from copy import copy
    canvas = ROOT.TCanvas()
    legend = ROOT.TLegend()
    [hist,leg,canvo,pad1] = drawHistograms(objectName,rootDataFile,normalize=False,legend=False,
                                               legendNames=runList,pads=True,drawOption="COLZ",maxColumns=3)
    fits = []
    myFunc = ROOT.TF1("myFunc","gaus",40,60)
    canvas.cd()
    for i,histo in enumerate(hist):
        try:
            histo.FitSlicesY(myFunc, 0, -1, 0, "QNR")
            fit = copy(ROOT.gDirectory.Get(histo.GetName()+"_1"))
        except:
            print("Could not perform fit.")
            return [],legend,canvas
        fit.SetYTitle("Gaus Fit Mean "+ histo.GetYaxis().GetTitle())
        fit.SetXTitle(histo.GetXaxis().GetTitle())
        fit.SetTitle(quant.GetYaxis().GetTitle()+" vs. "+quant.GetXaxis().GetTitle())
        fit.SetLineWidth(2)
        fits.append(fit)
        legend.AddEntry(fit,runList[i])
        fit.Draw("SAME L")
    legend.Draw()
    return fits, legend, canvas

In [ ]:
ROOT.gStyle.SetPalette(57)
ROOT.gStyle.SetPalette(55)
ROOT.gStyle.SetGridStyle(3)
ROOT.gStyle.SetGridWidth(1)
ROOT.gStyle.SetOptStat(0)
cols = list(ROOT.TColor.GetPalette())
def updateColors(histograms,palette):
    global runList_first, runList_comparison
    nFirst = len(runList_first)
    nComparison = len(runList_comparison)
    colors = []
    for i, hist in enumerate(histograms):
        if i < nFirst:
            color = ROOT.kBlue + i -round(nFirst/2)
        else:
            color = ROOT.kRed+i-nFirst-round(nComparison/2)
            hist.SetLineStyle(2)
        colors.append(color)
        hist.SetLineColor(color)
        hist.SetMarkerColor(color)
    return colors

def setHistRangesNormalized(trends):
    if type(trends[0]) in (ROOT.TH1D, ROOT.TH1F, ROOT.TH1C):
        # Initialize variables to track the min and max range based on entries
        maxRange = -float('inf')
        minRange = float('inf')
        
        # Iterate over each histogram
        for trend in trends:
            # Get the total number of bins
            nBins = trend.GetNbinsX()
            
            # Iterate over each bin to get the number of entries
            for i in range(1, nBins + 1):
                binEntries = trend.GetBinContent(i)
                if binEntries != 0:
                    maxRange = max(maxRange, binEntries)
                    minRange = min(minRange, binEntries)
        
        diff = maxRange - minRange
        
        # Set the range for each histogram
        for trend in trends:
            if len(trends) > 1:
                trend.SetMaximum(maxRange + diff * 0.3)
                trend.SetMinimum(minRange - diff * 0.3)
    else:
        raise Exception("Expected TH1s")


In [ ]:
# NOTEBOOK VARIABLES -- CHANGE THESE TO RUN FOR DIFFERENT PERIODS
debug = False

# First production in comparison
path = "myPath"
periodName_first = "myPeriod"
passName_first = "myPassName" # keep empty ("") if MC
    
# Second production in comparison
path_comparison = "myComparisonPath"
periodName_comparison = "myComparisonPeriod"
passName_comparison = "myComparisonPassName" # keep empty ("") if MC

if debug:
    print(path+"/"+periodName_first+"/"+passName_first+"/")
    print(path_comparison+"/"+periodName_comparison+"/"+passName_comparison+"/")

In [ ]:
if False:
    # First production in comparison
    path = "/cave/alice-tpc-qc/data/2022/"
    periodName_first = "LHC22o"
    passName_first = "apass7_skimmed" # keep empty ("") if MC
        
    # Second production in comparison
    path_comparison = "/cave/alice-tpc-qc/data/2023/"
    periodName_comparison = "LHC23y"
    passName_comparison = "apass4_skimmed" # keep empty ("") if MC
    
    if debug:
        print(path+"/"+periodName_first+"/"+passName_first+"/")
        print(path_comparison+"/"+periodName_comparison+"/"+passName_comparison+"/")

In [ ]:
passName = periodName_first+" vs "+periodName_comparison
# Find runs
fileList_first = glob.glob(path+"/"+periodName_first+"/"+passName_first+"/"+"*_QC.root")
fileList_first = [file for file in fileList_first if file[-13] != "_"]
fileList_first.sort()
runList_first = [fileList_first[i][-14:-8] for i in range(len(fileList_first))]
myRuns = ["526713","526714","526715","526716","526719","526720","526776","526860","526865","526886","526963","526964","526966","526967","526968","527015","527016","527028","527031","527033","527034","527038","527039","527041","527057","527076","527108","527109"]
runList_first = [run for run in runList_first]# if run in myRuns]


if debug : print(periodName_first, passName_first, "Files: ",fileList_first,"\n")
if debug : print(periodName_first, passName_first, "Runs: ", runList_first,"\n")
    
# Find runs in period to be compared
fileList_comparison = glob.glob(path_comparison+"/"+periodName_comparison+"/"+passName_comparison+"/"+"*_QC.root")
fileList_comparison = [file for file in fileList_comparison if file[-13] != "_"]
fileList_comparison.sort()
fileList_comparison = [file for file in fileList_comparison]
runList_comparison = [fileList_comparison[i][-14:-8] for i in range(len(fileList_comparison))]

if debug : print(periodName_comparison, passName_comparison, "Runs: ", fileList_comparison,"\n")
if debug : print(periodName_comparison, passName_comparison, "Runs: ", runList_comparison,"\n")
    
# Make sure only the same runs are there
runList_temp = [run for run in runList_first if run in runList_comparison]
if debug : print(periodName_comparison, passName_comparison, "Runs temp: ", runList_temp,"\n")
runList_first = runList_temp
fileList_first = [file for file in fileList_first if file[-14:-8] in runList_first]
fileList_comparison = [fileList_comparison[i] for i in range(len(fileList_comparison)) if runList_comparison[i] in runList_first]
runList_comparison = runList_first

# Read the Root Files
rootDataFile_first = [ROOT.TFile.Open(file,"READ") for file in fileList_first]   
rootDataFile_comparison = [ROOT.TFile.Open(file,"READ") for file in fileList_comparison]

# Combine files
rootDataFile = rootDataFile_first + rootDataFile_comparison
runList = [periodName_first +"_"+ run for run in runList_first] + [periodName_comparison +"_"+ run for run in runList_comparison]

In [ ]:
display(Markdown(f"# TPC Async QC Report - {periodName_first} {passName_first} vs Anchored MC {periodName_comparison}"))

**RCT Tables:**
+ [Bookkeeping RCT Table](https://ali-bookkeeping.cern.ch/?page=lhc-period-overview)
+ [2022](https://docs.google.com/spreadsheets/d/14vXFYVx3oVE8wgJKmIBhjvAt6NpGp7D6H4AmBM9E0Cw/edit#gid=0), [2023](https://docs.google.com/spreadsheets/d/1YBQLXWwwc3aC3B_PYVpFkTgEP0n6u1ovtYfiCOMWnTc/edit?pli=1#gid=0), [2023_PbPb](https://docs.google.com/spreadsheets/d/1vsl-pkLdUoNXZm6muPyMENWdDSKM7UDtQLcXc2B9QhE/edit#gid=492527515)

In [ ]:
#print("Period: "+periodName +" - Runs:")
print(periodName_first+":\t",runList_first)
print(periodName_comparison+":\t",runList_comparison)

In [ ]:
import datetime
now = datetime.datetime.now()
display(Markdown("+ Report automatically created on "+now.strftime("%d.%m.%Y at %H:%M:%S")))

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Plots" data-toc-modified-id="Plots-1">Plots</a></span><ul class="toc-item"><li><span><a href="#Tracks" data-toc-modified-id="Tracks-1.1">Tracks</a></span><ul class="toc-item"><li><span><a href="#Geometrical-distributions-of-tracks" data-toc-modified-id="Geometrical-distributions-of-tracks-1.1.1">Geometrical distributions of tracks</a></span></li><li><span><a href="#Track-properties" data-toc-modified-id="Track-properties-1.1.2">Track properties</a></span></li></ul></li><li><span><a href="#DCAr" data-toc-modified-id="DCAr-1.2">DCAr</a></span></li><li><span><a href="#PID" data-toc-modified-id="PID-1.3">PID</a></span><ul class="toc-item"><li><span><a href="#dEdxTot-of-MIP" data-toc-modified-id="dEdxTot-of-MIP-1.3.1">dEdxTot of MIP</a></span></li><li><span><a href="#TPC-Gain-calibration" data-toc-modified-id="TPC-Gain-calibration-1.3.2">TPC Gain calibration</a></span></li></ul></li></ul></li></ul></div>

---
## Plots

In [ ]:
ROOT.gStyle.SetPalette(57)
ROOT.gStyle.SetGridStyle(3)
ROOT.gStyle.SetGridWidth(1)
ROOT.gStyle.SetPalette(57)

---
### Tracks
#### Geometrical distributions of tracks
**Phi distribution A-side tracks:**

In [ ]:
%jsroot on
objectName="hPhiAside"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,
                                               normalize=True,legend=True,legendNames=runList,pads=False,
                                               maxColumns=2,drawOption="SAME L",yAxisRange=[2.2e-3,3.2e-3],
                                               grid=True)
    setHistRangesNormalized(hist)
    #Sector boundatries
    pad1.cd()
    sectorBoundary = ROOT.TLine(0,hist[0].GetMinimum(),0,hist[0].GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,hist[0].GetMinimum(),(j*2*math.pi)/18,hist[0].GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,hist[0].GetMaximum()*0.95+hist[0].GetMinimum()*0.05,str(j))
    pad1.SetGridy(1)
    canvas.SetCanvasSize(1200,400)
    updateColors(hist,0)
    canvas.Draw()

**Phi distribution C-side tracks:**

In [ ]:
%jsroot on
objectName="hPhiCside"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,
                                               normalize=True,legend=True,legendNames=runList,pads=False,
                                               maxColumns=2,drawOption="SAME L",yAxisRange=[2.2e-3,3.2e-3],
                                               grid=True)
    setHistRangesNormalized(hist)
    #Sector boundatries
    pad1.cd()
    sectorBoundary = ROOT.TLine(0,hist[0].GetMinimum(),0,hist[0].GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,hist[0].GetMinimum(),(j*2*math.pi)/18,hist[0].GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,hist[0].GetMaximum()*0.95+hist[0].GetMinimum()*0.05,str(j))
    pad1.SetGridy(1)
    canvas.SetCanvasSize(1200,400)
    updateColors(hist,0)
    canvas.Draw()

**Eta distribution:**

In [ ]:
%jsroot on
objectName="hEta"
xRange = [-1.1,1.1]
yRange = [2e-3,13e-3]
log  = "none"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,
                                               normalize=True,legend=True,legendNames=runList,pads=False,
                                               maxColumns=2,drawOption="SAME L",xAxisRange=xRange,yAxisRange=yRange,log=log,
                                               grid=True)
    setHistRangesNormalized(hist)
    pad1.SetGrid(1)
    canvas.SetCanvasSize(800,500)
    updateColors(hist,0)
    canvas.Draw()

In [ ]:
%jsroot on
objectName="h2DEtaPhi"
if checkIfExists(rootDataFile,objectName):
    c = ROOT.TCanvas("","",1200,500)
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas = c.cd()
    canvas.SetGridy(1)
    setHistRangesNormalized(quants)
    for quant in quants:
        quant.Draw("L SAME")
        #quant.GetYaxis().SetRangeUser(-.1,.1)
        quant.SetTitle(passName+" - Median eta vs phi")
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    legend.Draw()
    c.Draw()


#### Track properties
**Transverse momentum**

In [ ]:
%jsroot on
objectName="hPt"
xRange = [0,0]
yRange = [0,0]
log  = "logxy"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,
                                               normalize=True,legend=True,legendNames=runList,pads=False,
                                               maxColumns=2,drawOption="SAME L",xAxisRange=xRange,yAxisRange=yRange,log=log,
                                               grid=True)
    setHistRangesNormalized(hist)
    pad1.SetGrid(1)
    canvas.SetCanvasSize(800,500)
    updateColors(hist,0)
    canvas.Draw()

**Track sign**

In [ ]:
%jsroot on
objectName="hSign"
xRange = [0,0]
yRange = [0,0]
log  = "none"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,
                                               normalize=True,legend=True,legendNames=runList,pads=False,
                                               maxColumns=2,drawOption="SAME HIST",xAxisRange=xRange,yAxisRange=yRange,log=log,
                                               grid=True)
    setHistRangesNormalized(hist)
    pad1.SetGrid(1)
    canvas.SetCanvasSize(800,400)
    updateColors(hist,0)
    canvas.Draw()

**Charge / transverse momentum**

In [ ]:
%jsroot on
objectName="hQOverPt"
xRange = [0,0]
yRange = [0,0]
log  = "none"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,
                                               normalize=True,legend=True,legendNames=runList,pads=False,
                                               maxColumns=2,drawOption="SAME L",xAxisRange=xRange,yAxisRange=yRange,log=log,
                                               grid=True)
    setHistRangesNormalized(hist)
    pad1.SetGrid(1)
    canvas.SetCanvasSize(800,400)
    updateColors(hist,0)
    canvas.Draw()

**Number of clusters per track**

In [ ]:
%jsroot on
objectName="hNClustersAfterCuts"
xRange = [50,160]
yRange = [0,40e-3]
log  = "none"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,
                                               normalize=True,legend=True,legendNames=runList,pads=False,
                                               maxColumns=2,drawOption="SAME L",xAxisRange=xRange,yAxisRange=yRange,log=log,
                                               grid=True)
    setHistRangesNormalized(hist)
    pad1.SetGrid(1)
    canvas.SetCanvasSize(800,400)
    updateColors(hist,0)
    canvas.Draw()

**nClusters vs phi**

In [ ]:
%jsroot on
objectName="h2DNClustersPhiAside"
if checkIfExists(rootDataFile,objectName):
    c = ROOT.TCanvas("","",1200,500)
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas = c.cd()
    canvas.SetGridy(1)
    setHistRangesNormalized(quants)
    for quant in quants:
        quant.Draw("L SAME")
        #quant.GetYaxis().SetRangeUser(60,160)
        quant.SetTitle(passName+"- A-Side - nClusters per track vs phi")
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    legend.Draw()
    c.Draw()


In [ ]:
%jsroot on
objectName="h2DNClustersPhiCside"
if checkIfExists(rootDataFile,objectName):
    c = ROOT.TCanvas("","",1200,500)
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas = c.cd()
    canvas.SetGridy(1)
    setHistRangesNormalized(quants)
    for quant in quants:
        quant.Draw("L SAME")
        #quant.GetYaxis().SetRangeUser(60,160)
        quant.SetTitle(passName+"- C-Side - nClusters per track vs phi")
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    legend.Draw()
    c.Draw()


**nClusters vs eta**

In [ ]:
%jsroot on
objectName="h2DNClustersEta"
if checkIfExists(rootDataFile,objectName):
    c = ROOT.TCanvas("","",1200,500)
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas = c.cd()
    canvas.SetGrid(1)
    setHistRangesNormalized(quants)
    for quant in quants:
        quant.Draw("L SAME")
        quant.GetXaxis().SetRangeUser(-1.1,1.1)
        #quant.GetYaxis().SetRangeUser(60,160)
        quant.SetTitle(passName+"- nClusters per track vs eta")
    legend.Draw()
    c.Draw()


**nClusters vs transverse momentum**

In [ ]:
%jsroot on
objectName="h2DNClustersPt"
if checkIfExists(rootDataFile,objectName):
    c = ROOT.TCanvas("","",1200,500)
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas = c.cd()
    canvas.SetGrid(1)
    canvas.SetLogx()
    setHistRangesNormalized(quants)
    for quant in quants:
        quant.Draw("L SAME")
        #quant.GetYaxis().SetRangeUser(60,160)
        quant.SetTitle(passName+"- nClusters per track vs pT")
    legend.Draw()
    c.Draw()

### DCAr

In [ ]:
%jsroot on
objectName="hDCAr_A_Pos"
title = " - DCAr A-Side - Positive"
if checkIfExists(rootDataFile,objectName):
    c = ROOT.TCanvas("","",1200,500)
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas = c.cd()
    canvas.SetGridy(1)
    setHistRangesNormalized(quants)
    for quant in quants:
        quant.Draw("L SAME")
        #quant.GetYaxis().SetRangeUser(-3,3)
        quant.SetTitle(passName+title)
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    legend.Draw()
    c.Draw()

In [ ]:
%jsroot on
objectName="hDCAr_A_Neg"
title = " - DCAr A-Side - Negative"
if checkIfExists(rootDataFile,objectName):
    c = ROOT.TCanvas("","",1200,500)
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas = c.cd()
    canvas.SetGridy(1)
    setHistRangesNormalized(quants)
    for quant in quants:
        quant.Draw("L SAME")
        #quant.GetYaxis().SetRangeUser(-3,3)
        quant.SetTitle(passName+title)
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    legend.Draw()
    c.Draw()

In [ ]:
%jsroot on
objectName="hDCAr_C_Pos"
title = " - DCAr C-Side - Positive"
if checkIfExists(rootDataFile,objectName):
    c = ROOT.TCanvas("","",1200,500)
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas = c.cd()
    canvas.SetGridy(1)
    setHistRangesNormalized(quants)
    for quant in quants:
        quant.Draw("L SAME")
        #quant.GetYaxis().SetRangeUser(-3,3)
        quant.SetTitle(passName+title)
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    legend.Draw()
    c.Draw()

In [ ]:
%jsroot on
objectName="hDCAr_C_Neg"
title = " - DCAr C-Side - Negative"
if checkIfExists(rootDataFile,objectName):
    c = ROOT.TCanvas("","",1200,500)
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas = c.cd()
    canvas.SetGridy(1)
    setHistRangesNormalized(quants)
    for quant in quants:
        quant.Draw("L SAME")
        #quant.GetYaxis().SetRangeUser(-3,3)
        quant.SetTitle(passName+title)
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    legend.Draw()
    c.Draw()

### PID
#### dEdxTot of MIP

In [ ]:
%jsroot on
yRange = [0,0.09]
objects = ["hdEdxTotMIP_" + region for region in ["TPC","IROC","OROC1","OROC2","OROC3"]]
if all([checkIfExists(rootDataFile,objectName) for objectName in objects]):
    hists = []
    for objectName in objects:
        [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,legend=True,legendNames=runList,pads=False,
                                               drawOption="SAME L",yAxisRange=yRange,log="logx")
        updateColors(hist,cols)
        hists.append(copy(hist))
    c = ROOT.TCanvas("MIP","MIP",1100,600)
    c.Divide(2)
    leftPad = c.cd(1)
    leftPad.SetGrid(1)
    for hist_list in hists:
        setHistRangesNormalized(hist_list)
    for hist in hists[0]:
        hist.SetTitle(passName+" - "+hist.GetTitle())
        hist.Draw("SAME L")
        hist.SetStats(0)
    #Draw target 50
    targets = [ROOT.TLine(50,yRange[0],50,yRange[1]) for hist in hists]
    for i in range(len(objects)):
        targets[i].SetLineStyle(2)
        targets[i].SetLineColor(3)
        targets[i].SetLineWidth(3)
    targets[0].Draw()
    legend.Draw()
    rightPad = c.cd(2)
    rightPad.Divide(2,2)
    for i in range(1,5):
        rightPad.cd(i)
        ROOT.gPad.SetGrid(1)
        for hist in hists[i]:
            hist.SetTitle(passName+" - "+hist.GetTitle())
            hist.Draw("SAME L")
            hist.SetStats(0)
        targets[i].Draw()
    c.Draw()

#### TPC Gain calibration
**dEdx of MIP vs sector**

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsSec_TPC"
title = "TPC Mean MIP Q_{Tot} vs Sector"
if checkIfExists(rootDataFile,objectName):
    c = ROOT.TCanvas("","",1200,500)
    [quants,legend,canvas] = sliceAndFit(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas = c.cd()
    canvas.SetGridy(1)
    setHistRangesNormalized(quants)
    for quant in quants:
        quant.Draw("L SAME")
        #quant.GetYaxis().SetRangeUser(45,55)
        quant.SetTitle(passName+" - "+title)
    legend.Draw()    
    c.Draw()

**dEdx of Mip vs nClusters**

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsNcl_TPC"
title = "TPC Mean MIP Q_{Tot} vs nClusters"
if checkIfExists(rootDataFile,objectName):
    c = ROOT.TCanvas("","",1200,500)
    [quants,legend,canvas] = sliceAndFit(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas = c.cd()
    canvas.SetGridy(1)
    setHistRangesNormalized(quants)
    for quant in quants:
        quant.Draw("L SAME")
        #quant.GetYaxis().SetRangeUser(45,55)
        quant.GetXaxis().SetRangeUser(60,155)
        quant.SetTitle(passName+" - "+title)
    legend.Draw()
    c.Draw()

**dEdx of Mip vs tan(lambda)**

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsTgl_TPC"
title = "TPC Mean MIP Q_{Tot} vs tan(lambda)"
if checkIfExists(rootDataFile,objectName):
    c = ROOT.TCanvas("","",1200,500)
    [quants,legend,canvas] = sliceAndFit(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas = c.cd()
    canvas.SetGridy(1)
    setHistRangesNormalized(quants)
    for quant in quants:
        quant.Draw("L SAME")
        #quant.GetYaxis().SetRangeUser(45,55)
        quant.GetXaxis().SetRangeUser(-2,2)
        quant.SetTitle(passName+" - "+title)
    legend.Draw()
    c.Draw()

**dEdx of Mip vs sin(phi)**

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsSnp_TPC"
title = "TPC Mean MIP Q_{Tot} vs sin(phi)"
if checkIfExists(rootDataFile,objectName):
    c = ROOT.TCanvas("","",1200,500)
    [quants,legend,canvas] = sliceAndFit(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas = c.cd()
    canvas.SetGridy(1)
    setHistRangesNormalized(quants)
    for quant in quants:
        quant.Draw("L SAME")
        #quant.GetYaxis().SetRangeUser(45,55)
        quant.GetXaxis().SetRangeUser(-1,1)
        quant.SetTitle(passName+" - "+title)
    legend.Draw()
    c.Draw()